<a href="https://colab.research.google.com/github/gouravgujariya/consultjob_ml_project/blob/main/Deploying_Machine_Learning_model_using_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import string
import numpy as np
import pandas as pd
import random
import missingno
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn.metrics import accuracy_score, recall_score, plot_confusion_matrix

from wordcloud import WordCloud

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import warnings
warnings.filterwarnings("ignore")

In [40]:
data = pd.read_csv('/content/drive/MyDrive/consultjob/fake_job_postings.csv',engine='python', error_bad_lines=False)

Skipping line 4526: unexpected end of data


In [41]:
columns=['job_id', 'telecommuting', 'has_company_logo', 'has_questions', 'salary_range', 'employment_type']
for col in columns:
    del data[col]

data.fillna(' ', inplace=True)

In [42]:
def split(location):
    l = location.split(',')
    return l[0]

data['country'] = data.location.apply(split)

In [43]:
data['text']=data['title']+' '+data['location']+' '+data['company_profile']+' '+data['description']+' '+data['requirements']+' '+data['benefits']
del data['title']
del data['location']
del data['department']
del data['company_profile']
del data['description']
del data['requirements']
del data['benefits']
del data['required_experience']
del data['required_education']
del data['industry']
del data['function']
del data['country']

In [44]:
fraudjobs_text = data[data.fraudulent==1].text
actualjobs_text = data[data.fraudulent==0].text

In [45]:
# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [46]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [47]:
# creating our bag of words
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,3))

In [48]:
X_train, X_test, y_train, y_test = train_test_split(data.text, data.fraudulent, test_size=0.3)

In [22]:
clf_rf = RandomForestClassifier()

# Create pipeline using Bag of Words
pipe_rf = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', clf_rf)])

# fitting our model.
pipe_rf.fit(X_train,y_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x7fb2add61750>),
                ('vectorizer',
                 CountVectorizer(ngram_range=(1, 3),
                                 tokenizer=<function spacy_tokenizer at 0x7fb2addd4200>)),
                ('classifier', RandomForestClassifier())])

In [53]:
X_test[[4309]]

4309    Sales Representative   We are Netguru and we l...
Name: text, dtype: object

Saving the trained model

In [23]:
import pickle

In [24]:
filename = 'rf_trained_model.sav'
pickle.dump(pipe_rf, open(filename, 'wb'))

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
# loading the saved model
loaded_model = pickle.load(open('/content/rf_trained_model.sav', 'rb'))

In [27]:
scaled_inputs_all.iloc[shuffled_indices].values 

NameError: ignored

In [28]:
data.drop(index=list1,labels=None, axis=0, inplace=True,columns=None, level=None, errors='raise')

NameError: ignored

In [29]:
data.reset_index(drop=True, inplace=True)

In [30]:
prediction = loaded_model.predict(X_test[[1]])
print(prediction)

KeyError: ignored

In [32]:
import joblib

In [33]:
filename = 'finalized_model.sav'
joblib.dump(pipe_rf, filename)

['finalized_model.sav']

In [35]:
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test,y_test)
print(result)

0.9911634756995582


In [ ]:
loaded_model.predict([[292,Financial Accountant,"US, NY, New York",Finance,,"Shapeways is the leading 3D printing marketplace and community, empowering designers to bring amazing products to life. By giving anyone the ability to quickly and affordably turn ideas from digital designs into real products, Shapeways is fundamentally changing how products are made and by whom.Through Shapeways, designers gain access to the best industrial 3D printing technology, capable of manufacturing products with complex designs in a wide range of high-quality materials. 3D printing turns raw materials into original products, from wedding rings to rocketships, model trains to iPhone cases, and prototypes to industrial engineering parts. The Shapeways community can sell their products, share ideas, and get feedback from creative consumers and other designers around the world.Headquartered in New York, Shapeways has factories and offices in Eindhoven, Queens, and Seattle. Shapeways is a spin-out of the lifestyle incubator of Royal Philips Electronics, and our investors include Union Square Ventures, Index Ventures, Lux Capital, and Andreessen Horowitz.","Shapeways is a fast-growing, global startup, with the goal of making 3D printing more accessible and affordable. Everything we make is custom, tailored to our customer's wishes. Want to help deliver excellence to our growing community? You'll be the one making sure that every customer order is printed to perfection.Position SummaryWe are looking for a whip-smart, agile, Financial Accountant to join our growing finance team at Shapeways. The Financial Accountant will focus on the large volume of transactional accounting that our business generates daily. Our goal is to provide a high level of service to the company, ensuring that the people who rely on us (our colleagues, our suppliers, our business partners, our directors) have access to timely, accurate information about our business. Everything we do is geared towards making sure that the finance department runs efficiently and that the books and records of the corporation are accurate. The department is fast, nimble and adaptable to a growing, changing environment where new challenges present themselves on a regular basis.ResponsibilitiesAssist with all payables and refund processing as requiredFinancial statement preparation and analysisCorporate credit card accounting and ownership of programGeneral ledger accountingGeneral ledger system design and implementationShop-owner markup reporting and paymentsBudget to actual variance analysisFixed asset accountingSales taxCash reconciliationsAccrual analysis and preparationPayroll accountingMulti-currency accounting","A four year degree in Accounting3-5 years of direct experience working in an Accounting capacityAdvanced understanding of Microsoft Excel (pivot tables, VBA, macros, etc.)Experience with Intaact and/or NetSuite financialsExcellent communication and collaboration skillsAbility to thrive in a fast-paced, entrepreneurial, startup environmentFlexibility to work late/come in early as neededKnowledge of SQL and experience with an e-commerce business model a big plus","Medical Benefits (Individual medical benefits covered at 100%)Dental BenefitsVision insuranceFlexible Spending AccountCommuter Benefits through Zenefits (eligible from day one-allows employee to set aside pre-tax dollars for eligible commuting expenses)- represents an annual savings of 20-40%401K (eligible after 3 months at Shapeways)Four weeks of PTO (20 days total/year) in addition to national holidaysFive sick days/yearAdditional benefits:Short-term disability insurance through GuardianLong-term disability insurance through GuardianLife insurance through GuardianWhy join our team?Shapeways is breaking new ground in the field of 3D printing. With our website and marketplace for designers, our goal is to give everyone access to the revolution of additive manufacturing. This isn’t your typical web business – we have a physical product at the back end. If you’d like to work in a dynamic, collaborative, respectful environment where colleagues share ideas and encourage each other to think creatively, Shapeways is just what you’re looking for. Fun is also part of our DNA – a sense of humor is a must! You’ll also have the coolest desk toys anywhere.",0,1,0,Full-time,Associate,Bachelor's Degree,Consumer Goods,Accounting/Auditing,0
]])